In [2]:
"""
SpaceX Falcon 9 - Interactive Dashboard with Plotly Dash
Creating interactive visualizations for launch analysis
"""

# Install required packages
!pip install dash plotly pandas

import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Load data
df = pd.read_csv('spacex_clean.csv')
df = df.dropna(subset=['LaunchSiteName', 'Success'])

print("="*70)
print("SPACEX INTERACTIVE DASHBOARD")
print("="*70)
print(f"Data loaded: {len(df)} records\n")

# ============================================
# VISUALIZATION 1: Success Count by Site (Pie Chart)
# ============================================
print("Creating Visualization 1: Launch Success Count by Site...")

site_counts = df['LaunchSiteName'].value_counts()
fig1 = px.pie(
    values=site_counts.values,
    names=site_counts.index,
    title='Total Launch Count for All Sites',
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig1.update_traces(textposition='inside', textinfo='percent+label+value')
fig1.write_html('dashboard1_all_sites.html')
print("✅ Visualization 1 saved: dashboard1_all_sites.html")

# Display in notebook
fig1.show()

# ============================================
# VISUALIZATION 2: Best Site Success Rate (Pie Chart)
# ============================================
print("\nCreating Visualization 2: Best Site Success Rate...")

# Calculate success rate by site
site_success = df.groupby('LaunchSiteName')['Success'].agg(['sum', 'count'])
site_success['rate'] = site_success['sum'] / site_success['count']
best_site = site_success['rate'].idxmax()

print(f"Best site: {best_site} with {site_success.loc[best_site, 'rate']:.1%} success rate")

# Filter data for best site
df_best = df[df['LaunchSiteName'] == best_site]
success_counts = df_best['Success'].value_counts()

fig2 = px.pie(
    values=success_counts.values,
    names=['Success', 'Failure'],
    title=f'Success Rate at {best_site}',
    color_discrete_map={1:'#66BB6A', 0:'#EF5350'}
)
fig2.update_traces(textposition='inside', textinfo='percent+label+value')
fig2.write_html('dashboard2_best_site.html')
print("✅ Visualization 2 saved: dashboard2_best_site.html")

fig2.show()

# ============================================
# VISUALIZATION 3: Payload vs Outcome with Range Slider
# ============================================
print("\nCreating Visualization 3: Payload vs Outcome Scatter...")

# Create scatter plot
fig3 = px.scatter(
    df,
    x='PayloadMass',
    y='Success',
    color='LaunchSiteName',
    title='Payload Mass vs Launch Outcome',
    labels={
        'PayloadMass': 'Payload Mass (kg)',
        'Success': 'Launch Outcome (0=Failure, 1=Success)',
        'LaunchSiteName': 'Launch Site'
    },
    hover_data=['FlightNumber', 'Date', 'Orbit'],
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Add range slider
fig3.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=True),
        type="linear"
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=[0, 1],
        ticktext=['Failure', 'Success']
    ),
    height=600
)

fig3.write_html('dashboard3_payload_scatter.html')
print("✅ Visualization 3 saved: dashboard3_payload_scatter.html")

fig3.show()

# ============================================
# CREATE INTERACTIVE DASH APP
# ============================================
print("\nCreating Interactive Dash Application...")

# Initialize Dash app
app = dash.Dash(__name__)

# Get unique sites for dropdown
sites = ['ALL'] + sorted(df['LaunchSiteName'].unique().tolist())

# App layout
app.layout = html.Div([
    html.H1('SpaceX Launch Dashboard',
            style={'textAlign': 'center', 'color': '#2E7D32'}),

    html.Div([
        html.Label('Select Launch Site:'),
        dcc.Dropdown(
            id='site-dropdown',
            options=[{'label': site, 'value': site} for site in sites],
            value='ALL',
            placeholder='Select a launch site'
        )
    ], style={'width': '50%', 'margin': '20px auto'}),

    html.Div([
        dcc.Graph(id='success-pie-chart')
    ]),

    html.Div([
        html.Label('Select Payload Range (kg):'),
        dcc.RangeSlider(
            id='payload-slider',
            min=0,
            max=int(df['PayloadMass'].max()),
            step=500,
            marks={i: f'{i}kg' for i in range(0, int(df['PayloadMass'].max())+1, 2000)},
            value=[0, int(df['PayloadMass'].max())],
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'margin': '40px'}),

    html.Div([
        dcc.Graph(id='success-payload-scatter')
    ]),

    html.Div([
        html.H3('Dashboard Statistics', style={'textAlign': 'center'}),
        html.Div(id='stats-output', style={'textAlign': 'center', 'fontSize': '18px'})
    ])
])

# Callback for pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Show success count for all sites
        fig = px.pie(
            df,
            names='LaunchSiteName',
            title='Total Launch Success Count for All Sites',
            color_discrete_sequence=px.colors.qualitative.Set3
        )
    else:
        # Show success/failure for selected site
        filtered_df = df[df['LaunchSiteName'] == selected_site]
        success_counts = filtered_df['Success'].value_counts()
        fig = px.pie(
            values=success_counts.values,
            names=['Failure', 'Success'],
            title=f'Success vs Failure for {selected_site}',
            color_discrete_map={0:'#EF5350', 1:'#66BB6A'}
        )

    fig.update_traces(textposition='inside', textinfo='percent+label')
    return fig

# Callback for scatter plot
@app.callback(
    Output('success-payload-scatter', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter(selected_site, payload_range):
    # Filter by payload range
    filtered_df = df[
        (df['PayloadMass'] >= payload_range[0]) &
        (df['PayloadMass'] <= payload_range[1])
    ]

    # Filter by site if not ALL
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['LaunchSiteName'] == selected_site]

    # Create scatter plot
    fig = px.scatter(
        filtered_df,
        x='PayloadMass',
        y='Success',
        color='LaunchSiteName',
        title=f'Payload vs Outcome (Payload Range: {payload_range[0]}-{payload_range[1]} kg)',
        labels={
            'PayloadMass': 'Payload Mass (kg)',
            'Success': 'Launch Outcome',
            'LaunchSiteName': 'Launch Site'
        },
        hover_data=['FlightNumber', 'Date'],
        color_discrete_sequence=px.colors.qualitative.Set2
    )

    fig.update_layout(
        yaxis=dict(
            tickmode='array',
            tickvals=[0, 1],
            ticktext=['Failure', 'Success']
        )
    )

    return fig

# Callback for statistics
@app.callback(
    Output('stats-output', 'children'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_stats(selected_site, payload_range):
    # Filter data
    filtered_df = df[
        (df['PayloadMass'] >= payload_range[0]) &
        (df['PayloadMass'] <= payload_range[1])
    ]

    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['LaunchSiteName'] == selected_site]

    # Calculate statistics
    total = len(filtered_df)
    success = filtered_df['Success'].sum()
    failure = total - success
    success_rate = (success / total * 100) if total > 0 else 0

    return html.Div([
        html.P(f"Total Launches: {total}"),
        html.P(f"Successful: {success} | Failed: {failure}"),
        html.P(f"Success Rate: {success_rate:.1f}%",
               style={'color': '#2E7D32', 'fontWeight': 'bold'})
    ])

# ============================================
# FINAL SUMMARY
# ============================================
print("\n" + "="*70)
print("✅ DASHBOARD CREATION COMPLETED!")
print("="*70)
print("\nFILES CREATED:")
print("1. dashboard1_all_sites.html")
print("2. dashboard2_best_site.html")
print("3. dashboard3_payload_scatter.html")
print("\nTO RUN INTERACTIVE APP:")
print("Run: app.run_server(mode='inline', port=8050)")
print("="*70)

# Run the app (uncomment to run)
# app.run_server(mode='inline', port=8050, debug=True)

print("\n📸 INSTRUCTIONS FOR SCREENSHOTS:")
print("1. Run the app with: app.run_server(mode='inline')")
print("2. Take screenshots of each visualization")
print("3. Save as: dashboard1.png, dashboard2.png, dashboard3.png")
print("4. Insert screenshots into PowerPoint slides 39-41")


SPACEX INTERACTIVE DASHBOARD
Data loaded: 205 records

Creating Visualization 1: Launch Success Count by Site...
✅ Visualization 1 saved: dashboard1_all_sites.html



Creating Visualization 2: Best Site Success Rate...
Best site: KSC LC 39A with 94.8% success rate
✅ Visualization 2 saved: dashboard2_best_site.html



Creating Visualization 3: Payload vs Outcome Scatter...
✅ Visualization 3 saved: dashboard3_payload_scatter.html



Creating Interactive Dash Application...

✅ DASHBOARD CREATION COMPLETED!

FILES CREATED:
1. dashboard1_all_sites.html
2. dashboard2_best_site.html
3. dashboard3_payload_scatter.html

TO RUN INTERACTIVE APP:
Run: app.run_server(mode='inline', port=8050)

📸 INSTRUCTIONS FOR SCREENSHOTS:
1. Run the app with: app.run_server(mode='inline')
2. Take screenshots of each visualization
3. Save as: dashboard1.png, dashboard2.png, dashboard3.png
4. Insert screenshots into PowerPoint slides 39-41
